In [2]:
import pandas as pd
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from collections import defaultdict
import numpy as np
import re

In [3]:
movies=pd.read_csv("netflix_titles.csv",encoding='latin-1')
movies=movies[['title','description','listed_in','Mood']]
title_array=movies['title'].to_numpy()
# print(title_array)
movies.head()

,title,description,listed_in,Mood
0,3%,In a future where the elite inhabit an island ...,"International TV Shows, TV Dramas, TV Sci-Fi &...",Fearful
1,7:19,After a devastating earthquake hits Mexico Cit...,"Dramas, International Movies",Adventurous
2,23:59,"When an army recruit is found dead, his fellow...","Horror Movies, International Movies",Fearful
3,9,"In a postapocalyptic world, rag-doll robots hi...","Action & Adventure, Independent Movies, Sci-Fi...",Adventurous
4,46,A genetics professor experiments with a treatm...,"International TV Shows, TV Dramas, TV Mysteries",Adventurous


In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
tokenized_description=[]
#check if each string in word list contains any of the above - one has :detectiveâ\x80\x99s 
for index,row in movies.iterrows():
    words = nltk.word_tokenize(str(row['description']).lower())
    word_list=[w.strip() for w in words if re.match(r'[^\W\d]*$', w) and w not in stop_words and w.isalpha()]
    tokenized_description.append(word_list)
tokenized_array_desc=np.array(tokenized_description)

<ipython-input-4-8310012fbd82>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  tokenized_array_desc=np.array(tokenized_description)


In [5]:
# Lemmatize with POS Tag
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

for index,desc in enumerate(tokenized_array_desc):
    desc_lemmatized=[]
    for word,tag in nltk.pos_tag(desc):
        word_lemmatized=lemmatizer.lemmatize(word,tag_map[tag[0]])
        desc_lemmatized.append(word_lemmatized)
    movies.loc[index,'description_final']=str(desc_lemmatized)
    #This throws an error FIX IT
print(movies['description_final'])

0      ['future', 'elite', 'inhabit', 'island', 'para...
1      ['devastate', 'earthquake', 'hit', 'mexico', '...
2      ['army', 'recruit', 'find', 'dead', 'fellow', ...
3      ['postapocalyptic', 'world', 'robot', 'hide', ...
4      ['genetics', 'professor', 'experiment', 'treat...
                             ...                        
916    ['recover', 'alcoholic', 'talal', 'wake', 'ins...
917    ['political', 'cartoonist', 'crime', 'reporter...
918    ['cruel', 'uncle', 'abandon', 'young', 'orphan...
919    ['live', 'alone', 'spooky', 'town', 'young', '...
920    ['look', 'survive', 'world', 'take', 'zombie',...
Name: description_final, Length: 921, dtype: object


In [6]:
X=movies[['description_final']]
Y=movies[['Mood']].to_numpy()

In [7]:
moodList = []
for index, row in movies.iterrows():
    moods = row["Mood"] 
    if moods not in moodList:
        moodList.append(moods)
print(moodList)

['Fearful', 'Adventurous', 'Romantic', 'Cheerful', ' Romantic']


In [8]:
def getBinary(mood_str):
    array=[]
    for mood in moodList:  
        if mood==mood_str:
            array.append(1)
        else:
            array.append(0)
    return array
for i in moodList:
    print(getBinary(i))

[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]


In [9]:
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
movies['Mood_bin'] = movies['Mood'].apply(lambda r:getBinary(r))
movies

,title,description,listed_in,Mood,description_final,Mood_bin
0,3%,In a future where the elite inhabit an island ...,"International TV Shows, TV Dramas, TV Sci-Fi &...",Fearful,"['future', 'elite', 'inhabit', 'island', 'para...","[1, 0, 0, 0, 0]"
1,7:19,After a devastating earthquake hits Mexico Cit...,"Dramas, International Movies",Adventurous,"['devastate', 'earthquake', 'hit', 'mexico', '...","[0, 1, 0, 0, 0]"
2,23:59,"When an army recruit is found dead, his fellow...","Horror Movies, International Movies",Fearful,"['army', 'recruit', 'find', 'dead', 'fellow', ...","[1, 0, 0, 0, 0]"
3,9,"In a postapocalyptic world, rag-doll robots hi...","Action & Adventure, Independent Movies, Sci-Fi...",Adventurous,"['postapocalyptic', 'world', 'robot', 'hide', ...","[0, 1, 0, 0, 0]"
4,46,A genetics professor experiments with a treatm...,"International TV Shows, TV Dramas, TV Mysteries",Adventurous,"['genetics', 'professor', 'experiment', 'treat...","[0, 1, 0, 0, 0]"
...,...,...,...,...,...,...
916,Zinzana,Recovering alcoholic Talal wakes up inside a s...,"Dramas, International Movies, Thrillers",Fearful,"['recover', 'alcoholic', 'talal', 'wake', 'ins...","[1, 0, 0, 0, 0]"
917,Zodiac,"A political cartoonist, a crime reporter and a...","Cult Movies, Dramas, Thrillers",Fearful,"['political', 'cartoonist', 'crime', 'reporter...","[1, 0, 0, 0, 0]"
918,Zokkomon,"When his cruel uncle abandons him, a young orp...","Children & Family Movies, Dramas",Adventurous,"['cruel', 'uncle', 'abandon', 'young', 'orphan...","[0, 1, 0, 0, 0]"
919,Zombie Dumb,"While living alone in a spooky town, a young g...","Kids' TV, Korean TV Shows, TV Comedies",Cheerful,"['live', 'alone', 'spooky', 'town', 'young', '...","[0, 0, 0, 1, 0]"


In [18]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [20]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [71]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for desc in movies['description_final']:
    str1 = desc.replace(']','').replace('[','')
    l = str1.replace("'",'').strip().replace(" ","").split(",")
    for d in l:
        if d not in vocab:
            vocab[d]=index
            index+=1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'future': 1, 'elite': 2, 'inhabit': 3, 'island': 4, 'paradise': 5, 'far': 6, 'crowd': 7, 'slum': 8, 'get': 9, 'one': 10, 'chance': 11, 'join': 12, 'save': 13, 'squalor': 14, 'devastate': 15, 'earthquake': 16, 'hit': 17, 'mexico': 18, 'city': 19, 'trap': 20, 'survivor': 21, 'walk': 22, 'life': 23, 'wait': 24, 'rescue': 25, 'try': 26, 'desperately': 27, 'stay': 28, 'alive': 29, 'army': 30, 'recruit': 31, 'find': 32, 'dead': 33, 'fellow': 34, 'soldier': 35, 'force': 36, 'confront': 37, 'terrify': 38, 'secret': 39, 'haunting': 40, 'jungle': 41, 'training': 42, 'camp': 43, 'postapocalyptic': 44, 'world': 45, 'robot': 46, 'hide': 47, 'fear': 48, 'dangerous': 49, 'machine': 50, 'exterminate': 51, 'brave': 52, 'newcomer': 53, 'group': 54, 'genetics': 55, 'professor': 56, 'experiment': 57, 'treatment': 58, 'comatose': 59, 'sister': 60, 'blend': 61, 'medical': 62, 'shamanic': 63, 'cure': 64, 'unlocks': 65, 'shock': 66, 'side': 67, 'effect': 68, 'doctor': 69, 'go': 70, 'miss': 71, 'p

In [75]:
def str2Int(array):
    str1 = array.replace(']','').replace('[','')
    l = str1.replace("'",'').strip().replace(" ","").split(",")
    data=[]
    for word in l:
        if word in vocab:
            data.append(vocab[word])
    return data


In [76]:
movies['description_int'] = movies['description_final'].apply(lambda r:str2Int(r))
movies


,title,description,listed_in,Mood,description_final,Mood_bin,description_int
0,3%,In a future where the elite inhabit an island ...,"International TV Shows, TV Dramas, TV Sci-Fi &...",Fearful,"['future', 'elite', 'inhabit', 'island', 'para...","[1, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
1,7:19,After a devastating earthquake hits Mexico Cit...,"Dramas, International Movies",Adventurous,"['devastate', 'earthquake', 'hit', 'mexico', '...","[0, 1, 0, 0, 0]","[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 2..."
2,23:59,"When an army recruit is found dead, his fellow...","Horror Movies, International Movies",Fearful,"['army', 'recruit', 'find', 'dead', 'fellow', ...","[1, 0, 0, 0, 0]","[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4..."
3,9,"In a postapocalyptic world, rag-doll robots hi...","Action & Adventure, Independent Movies, Sci-Fi...",Adventurous,"['postapocalyptic', 'world', 'robot', 'hide', ...","[0, 1, 0, 0, 0]","[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 12, 54]"
4,46,A genetics professor experiments with a treatm...,"International TV Shows, TV Dramas, TV Mysteries",Adventurous,"['genetics', 'professor', 'experiment', 'treat...","[0, 1, 0, 0, 0]","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6..."
...,...,...,...,...,...,...,...
916,Zinzana,Recovering alcoholic Talal wakes up inside a s...,"Dramas, International Movies, Thrillers",Fearful,"['recover', 'alcoholic', 'talal', 'wake', 'ins...","[1, 0, 0, 0, 0]","[970, 4067, 4068, 1516, 90, 875, 155, 2203, 42..."
917,Zodiac,"A political cartoonist, a crime reporter and a...","Cult Movies, Dramas, Thrillers",Fearful,"['political', 'cartoonist', 'crime', 'reporter...","[1, 0, 0, 0, 0]","[709, 2010, 630, 437, 1789, 877, 956, 4070, 40..."
918,Zokkomon,"When his cruel uncle abandons him, a young orp...","Children & Family Movies, Dramas",Adventurous,"['cruel', 'uncle', 'abandon', 'young', 'orphan...","[0, 1, 0, 0, 0]","[2744, 2826, 3604, 188, 428, 2571, 1245, 4073,..."
919,Zombie Dumb,"While living alone in a spooky town, a young g...","Kids' TV, Korean TV Shows, TV Comedies",Cheerful,"['live', 'alone', 'spooky', 'town', 'young', '...","[0, 0, 0, 1, 0]","[228, 2163, 1256, 381, 188, 334, 1237, 4074, 9..."


In [58]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'future', 2: ' elite', 3: ' inhabit', 4: ' island', 5: ' paradise', 6: ' far', 7: ' crowd', 8: ' slum', 9: ' get', 10: ' one', 11: ' chance', 12: ' join', 13: ' save', 14: ' squalor', 15: 'devastate', 16: ' earthquake', 17: ' hit', 18: ' mexico', 19: ' city', 20: ' trap', 21: ' survivor', 22: ' walk', 23: ' life', 24: ' wait', 25: ' rescue', 26: ' try', 27: ' desperately', 28: ' stay', 29: ' alive', 30: 'army', 31: ' recruit', 32: ' find', 33: ' dead', 34: ' fellow', 35: ' soldier', 36: ' force', 37: ' confront', 38: ' terrify', 39: ' secret', 40: ' haunting', 41: ' jungle', 42: ' training', 43: ' camp', 44: 'postapocalyptic', 45: ' world', 46: ' robot', 47: ' hide', 48: ' fear', 49: ' dangerous', 50: ' machine', 51: ' exterminate', 52: ' brave', 53: ' newcomer', 54: ' group', 55: 'genetics', 56: ' professor', 57: ' experiment', 58: ' treatment', 59: ' comatose', 60: ' sister', 61: ' blend', 62: ' medical', 63: ' shamanic', 64: ' cure', 65: ' unlocks', 66: ' shock', 67:

In [3]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
    targets, contexts, labels = [], [], []

    # Build the sampling table for vocab_size tokens.
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
    for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
        context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
        negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

    # Build context and label vectors (for one target word)
    negative_sampling_candidates = tf.expand_dims(
      negative_sampling_candidates, 1)

    context = tf.concat([context_class, negative_sampling_candidates], 0)
    label = tf.constant([1] + [0]*num_ns, dtype="int64")

    # Append each element from the training example to global lists.
    targets.append(target_word)
    contexts.append(context)
    labels.append(label)

    return targets, contexts, labels

In [112]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
# Vectorize the text using CountVectorizer
count_vectorizer = CountVectorizer()
train_cv = count_vectorizer.fit_transform(movies['description_final']).toarray()
# test_cv = count_vectorizer.transform(Y)

## Keeping only non-zero elements to preserve space 
print(train_cv)
X=train_cv
Y=movies['Mood_pos']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2)
print(X_train.shape)
print(X_test.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]
(644, 4077)
(277, 4077)


In [92]:
maxLength=0
for d in movies['description_int']:
    if maxLength<len(d):
        maxLength=len(d)
print(maxLength)


26


In [97]:
from sklearn.preprocessing import MultiLabelBinarizer
def equalizedLength(arr,maxLength):
    while len(arr)<maxLength:
        arr.append(0)
    print(arr)
    return arr
movies['description_int'] = movies['description_int'].apply(lambda r:equalizedLength(r,maxLength))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 4, 42, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 12, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[69, 70, 71, 72, 73, 74, 75, 62, 76, 77, 78, 79, 80, 81, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[82, 73, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[94, 95, 96, 97, 73, 98, 99, 100, 101, 102, 103, 104, 105, 106, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[133, 134, 135, 136, 137, 138, 139, 140, 141, 142,

In [95]:
def getPosition(array):
    count=0
    for a in array:
#         print(a)
        if(int(a)==1):
            return count
        count+=1

movies['Mood_pos'] = movies['Mood_bin'].apply(lambda r:getPosition(r))
movies

,title,description,listed_in,Mood,description_final,Mood_bin,Mood_pos
0,3%,In a future where the elite inhabit an island ...,"International TV Shows, TV Dramas, TV Sci-Fi &...",Fearful,"['future', 'elite', 'inhabit', 'island', 'para...","[1, 0, 0, 0, 0]",0
1,7:19,After a devastating earthquake hits Mexico Cit...,"Dramas, International Movies",Adventurous,"['devastate', 'earthquake', 'hit', 'mexico', '...","[0, 1, 0, 0, 0]",1
2,23:59,"When an army recruit is found dead, his fellow...","Horror Movies, International Movies",Fearful,"['army', 'recruit', 'find', 'dead', 'fellow', ...","[1, 0, 0, 0, 0]",0
3,9,"In a postapocalyptic world, rag-doll robots hi...","Action & Adventure, Independent Movies, Sci-Fi...",Adventurous,"['postapocalyptic', 'world', 'robot', 'hide', ...","[0, 1, 0, 0, 0]",1
4,46,A genetics professor experiments with a treatm...,"International TV Shows, TV Dramas, TV Mysteries",Adventurous,"['genetics', 'professor', 'experiment', 'treat...","[0, 1, 0, 0, 0]",1
...,...,...,...,...,...,...,...
916,Zinzana,Recovering alcoholic Talal wakes up inside a s...,"Dramas, International Movies, Thrillers",Fearful,"['recover', 'alcoholic', 'talal', 'wake', 'ins...","[1, 0, 0, 0, 0]",0
917,Zodiac,"A political cartoonist, a crime reporter and a...","Cult Movies, Dramas, Thrillers",Fearful,"['political', 'cartoonist', 'crime', 'reporter...","[1, 0, 0, 0, 0]",0
918,Zokkomon,"When his cruel uncle abandons him, a young orp...","Children & Family Movies, Dramas",Adventurous,"['cruel', 'uncle', 'abandon', 'young', 'orphan...","[0, 1, 0, 0, 0]",1
919,Zombie Dumb,"While living alone in a spooky town, a young g...","Kids' TV, Korean TV Shows, TV Comedies",Cheerful,"['live', 'alone', 'spooky', 'town', 'young', '...","[0, 0, 0, 1, 0]",3


In [116]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
# y_transformed=mlb.fit_transform(Y_train)
# print(y_transformed)
# print(Y_train.flatten())
# print(x_transformed)
clf.fit(X_train,Y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(Y_test,y_pred))

0.6714801444043321


In [99]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
        context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
        negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

        # Build context and label vectors (for one target word)
        negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

        context = tf.concat([context_class, negative_sampling_candidates], 0)
        label = tf.constant([1] + [0]*num_ns, dtype="int64")

        # Append each element from the training example to global lists.
        targets.append(target_word)
        contexts.append(context)
        labels.append(label)

    return targets, contexts, labels